# Basic finmag tutorial

## Setting up a basic simulation

Depending on the functionalities required, different finmag modules should be imported. For the beginning, we import the following:

In [1]:
import dolfin as df  # enables us to create basic meshes
from finmag import Simulation as Sim
from finmag.energies import Exchange, DMI, Zeeman, Demag, UniaxialAnisotropy

[2015-11-13 13:33:38] INFO: Finmag logging output will be appended to file: '/home/mb4e10/.finmag/global.log'
[2015-11-13 13:33:38] DEBUG: Building modules in 'native'...
[2015-11-13 13:33:39] DEBUG: FinMag          3042844973cf14bc02c909b16ece9847aa26b637
[2015-11-13 13:33:39] DEBUG: Dolfin          1.6.0                Matplotlib      lazily loaded       
[2015-11-13 13:33:39] DEBUG: Numpy           1.10.1               Scipy           0.15.1              
[2015-11-13 13:33:39] DEBUG: IPython         4.0.0                Python          2.7.6               
[2015-11-13 13:33:39] DEBUG: Paraview        4.0.1-1ubuntu1       Sundials        2.5.0               
[2015-11-13 13:33:39] DEBUG: Boost-Python    <unknown>            Linux           Ubuntu 14.04.3 LTS  
[2015-11-13 13:33:39] DEBUG: Registering debug signal handler. Press Ctrl-Z any time to stop execution and jump into the debugger.


Firstly, we create a three-dimensional mesh with dimensions $l_{x} = 50$, $l_{y} = 50$, and $l_{z} = 10$. So, we need to provide two points as well as the mesh discretisation in all three directions ($n_{x}$, $n_{y}$, $n_{z}$).

In [2]:
lx = 50  # (m)
ly = 50  # (m)
lz = 10  # (m)

nx = 25  # number of vertices in x-direction
ny = 25
nz = 5

mesh = df.BoxMesh(df.Point(0, 0, 0), df.Point(lx, ly, lz), nx, ny, nz)

In the next step, we need to define some basic material parameters:

In [3]:
Ms = 1e6  # magnetisation saturation (A/m)
A = 1e-12  # exchange energy constant (J/m)
D = 1e-3  # DMI constant (J/m**2)
K = 1e5  # uniaxial anisotropy constant (J/m**3)
Kaxis = (0, 0, 1)  # uniaxial anisotropy axis
H = (0, 0, 1e7)  # external magnetic field (A/m)

Now, we can create a simulation object. The first argument is the mesh, the second one is the magnetisation saturation, and the third one is the unit length. Unit length should be specified because the mesh is initially created in meteres.

In [4]:
sim = Sim(mesh, Ms, unit_length=1e-9)

[2015-11-13 13:34:06] INFO: Finmag logging output will be written to file: '/home/mb4e10/finmag/doc/ipython_notebooks_src/robert/unnamed.log' (any old content will be overwritten).
[2015-11-13 13:34:06] DEBUG: Creating DataWriter for file 'unnamed.ndt'
[2015-11-13 13:34:06] INFO: Creating Sim object name='unnamed', instance_id=0 (rank=0/1).
[2015-11-13 13:34:06] DEBUG:    Total number of Sim objects in this session: 1
[2015-11-13 13:34:06] INFO: <Mesh of topological dimension 3 (tetrahedra) with 4056 vertices and 18750 cells, ordered>
/usr/local/lib/python2.7/dist-packages/aeon/timer.py:35: UserWarning: You are nesting measurements in __init__::LLG.
  warnings.warn("You are nesting measurements in {}::{}.".format(name, group))
[2015-11-13 13:34:07] DEBUG: Creating LLG object.


When the simulation object is creates, different interactions can be added to it:

In [5]:
sim.add(Exchange(A))  # add exchange interaction
sim.add(DMI(D))  # add DMI interaction
# sim.add(DMI(D, dmi_type='interfacial'))  # interfacial DMI interaction
sim.add(Zeeman(H))  # add Zeeman energy
sim.add(UniaxialAnisotropy(K, Kaxis))  # add uniaxial anisotropy
sim.add(Demag())  # add demagnetisation (magneostatic) energy

[2015-11-13 13:34:11] DEBUG: Creating Exchange object with method box-matrix-petsc, in Jacobian.
[2015-11-13 13:34:11] DEBUG: Adding interaction Exchange to simulation.
[2015-11-13 13:34:11] DEBUG: Creating DMI object with method box-matrix-petsc, in Jacobian.
[2015-11-13 13:34:11] DEBUG: Adding interaction DMI to simulation.
[2015-11-13 13:34:11] DEBUG: Adding interaction Zeeman to simulation.
[2015-11-13 13:34:11] DEBUG: Creating UniaxialAnisotropy object with method box-matrix-petsc, in Jacobian.
[2015-11-13 13:34:11] DEBUG: Adding interaction Anisotropy to simulation.
[2015-11-13 13:34:12] DEBUG: Creating Demag object with solver 'FK'.
[2015-11-13 13:34:12] DEBUG: Adding interaction Demag to simulation.
[2015-11-13 13:34:12] DEBUG: Using Krylov solver for demag.
[2015-11-13 13:34:12] DEBUG: Boundary element matrix uses 23.42 MB of memory.


So, at this point the Hamiltonian is created. Now, we can set parameters in the LLG equation. Precession and damping terms are enabled by default and other terms can be added separately.

In [6]:
sim.alpha = 0.5  # set the Gilbert damping

When both Hamiltonian and LLG equations are set, we need to set the intial magnetisation before we relax the system:

In [7]:
sim.set_m((1, 0, 0))

The system is relaxed:

In [8]:
sim.relax()

[2015-11-13 13:34:18] INFO: Simulation will run until relaxation of the magnetisation.
[2015-11-13 13:34:18] DEBUG: Relaxation parameters: stopping_dmdt=1.0 (degrees per nanosecond), dt_limit=1e-10, dmdt_increased_counter_limit=10
[2015-11-13 13:34:18] INFO: Creating integrator with backend sundials and arguments {'reltol': 1e-06, 'abstol': 1e-06}.
[2015-11-13 13:34:18] DEBUG: Updating get method for steps in TableWriter(name=unnamed.ndt)
[2015-11-13 13:34:18] DEBUG: Updating get method for last_step_dt in TableWriter(name=unnamed.ndt)
[2015-11-13 13:34:18] DEBUG: Updating get method for dmdt in TableWriter(name=unnamed.ndt)
/usr/local/lib/python2.7/dist-packages/aeon/timer.py:35: UserWarning: You are nesting measurements in compute_field::DMI.
  warnings.warn("You are nesting measurements in {}::{}.".format(name, group))
/usr/local/lib/python2.7/dist-packages/aeon/timer.py:35: UserWarning: You are nesting measurements in compute_field::UniaxialAnisotropy.
  warnings.warn("You are nest

The magnetisation data can be either:
  - shown as the numpy array
  - saved to the h5 file
  - saved as a pvd file so that it can be visualised in paraview or mayavi

In [9]:
print sim.llg.m_field.vector().array()  # show as a numpy array

[-0.03065838 -0.04465918 -0.0509022  ...,  0.99740855  0.99715546
  0.99694063]


In [10]:
# Save the magnetisation in the HDF5 file
h5_filename = 'example1'
sim.llg.m_field.save_hdf5(h5_filename, t=0)

Debug: (0/1) opening file example1.h5
Debug: (0/1) writing json file example1.json
Debug: (0/1) writing json file example1.json


A more detailed tutorial on saving and reading HDF5 files is provided separately.

In [11]:
# Save the VTK file for visualisation using Paraview or Mayavi
pvd_filename = 'relaxed_state.pvd'
sim.save_vtk(pvd_filename)

[2015-11-13 13:34:55] DEBUG: Saved field at t=1.97552250671e-11 to file 'relaxed_state.pvd' (snapshot #0; saving took 0.038 seconds).


## Multiple materials (spatially varying parameters)

If multiple materials are present in the system, spatially varying parameters should be provided. The simplest way of doing that is to define a Python function for these parameters. For instance, in the case of DMI:

In [12]:
def D_fun(pos):
    x, y, z = pos[0], pos[1], pos[2]
    if x < lx/2.:
        return -D
    else:
        return D

Now, this function can be added to the simulation, instead of D. For instance: sim.add(DMI(D_fun)). The same procedure applies for all parameters introduced so far.

## Extracting magnetisation data

The magnetisation at a single point can be sampled as:

In [13]:
sampling_point = (20, 15, 5)  # coordinates of a sampling point
sampled_magnetisation = sim.llg.m_field.probe(sampling_point)
print sampled_magnetisation

[ -5.85305001e-05   1.73350586e-04   9.99999836e-01]
